In [1]:
import numpy as np
import pandas as pd
import datetime as dt
from ast import literal_eval

In [2]:
pd.set_option("display.max_columns", None)
pd.set_option('display.max_colwidth', 100)
pd.options.display.float_format = '{:,.2f}'.format

In [3]:
df = pd.read_csv('data.csv')
df['date'] = pd.to_datetime(df.date)
df

,csn,date,transaction_info
0,Y2NgaWJoYw==,2018-03-02,"[{'article': '10020163', 'salesquantity': 2.0, 'price': 18250.0}]"
1,Y2NgaWJoYw==,2018-03-04,"[{'article': '10026562', 'salesquantity': 3.0, 'price': 13000.0}, {'article': '10320883', 'sales..."
2,Y2NgaWlpYA==,2018-02-27,"[{'article': '10013531', 'salesquantity': 1.0, 'price': 17800.0}, {'article': '10015613', 'sales..."
3,Y2NgaWlpYA==,2018-03-16,"[{'article': '10008991', 'salesquantity': 1.0, 'price': 57000.0}, {'article': '10010168', 'sales..."
4,Y2NgaWlpYA==,2018-05-24,"[{'article': '10009698', 'salesquantity': 1.0, 'price': 48900.0}, {'article': '10009701', 'sales..."
...,...,...,...
131789,amxpamJk,2018-03-24,"[{'article': '10324916', 'salesquantity': 1.0, 'price': 29700.0}, {'article': '10314277', 'sales..."
131790,amxpamJk,2018-04-07,"[{'article': '10012785', 'salesquantity': 1.0, 'price': 48000.0}, {'article': '10019005', 'sales..."
131791,amxpamJk,2018-04-21,"[{'article': '10083185', 'salesquantity': 1.0, 'price': 7000.0}, {'article': '10053361', 'salesq..."
131792,amxpamJk,2018-05-20,"[{'article': '10054789', 'salesquantity': 0.506, 'price': 38899.21}, {'article': '10053664', 'sa..."


In [4]:
df['transaction_info'] = df['transaction_info'].apply(literal_eval) #convert to list type
df = df.explode('transaction_info')
df

,csn,date,transaction_info
0,Y2NgaWJoYw==,2018-03-02,"{'article': '10020163', 'salesquantity': 2.0, 'price': 18250.0}"
1,Y2NgaWJoYw==,2018-03-04,"{'article': '10026562', 'salesquantity': 3.0, 'price': 13000.0}"
1,Y2NgaWJoYw==,2018-03-04,"{'article': '10320883', 'salesquantity': 2.0, 'price': 43000.0}"
2,Y2NgaWlpYA==,2018-02-27,"{'article': '10013531', 'salesquantity': 1.0, 'price': 17800.0}"
2,Y2NgaWlpYA==,2018-02-27,"{'article': '10015613', 'salesquantity': 1.0, 'price': 5600.0}"
...,...,...,...
131792,amxpamJk,2018-05-20,"{'article': '10225636', 'salesquantity': 0.182, 'price': 51500.0}"
131793,amxpbWNk,2018-04-11,"{'article': '10016399', 'salesquantity': 2.0, 'price': 57700.0}"
131793,amxpbWNk,2018-04-11,"{'article': '10016834', 'salesquantity': 1.0, 'price': 97000.0}"
131793,amxpbWNk,2018-04-11,"{'article': '10018739', 'salesquantity': 1.0, 'price': 82500.0}"


In [5]:
df = pd.concat([df.reset_index(drop=True), pd.DataFrame(list(df.transaction_info)).reset_index()], axis=1)
df = df.drop(columns=['transaction_info'])
df

,csn,date,index,article,salesquantity,price
0,Y2NgaWJoYw==,2018-03-02,0,10020163,2.00,"18,250.00"
1,Y2NgaWJoYw==,2018-03-04,1,10026562,3.00,"13,000.00"
2,Y2NgaWJoYw==,2018-03-04,2,10320883,2.00,"43,000.00"
3,Y2NgaWlpYA==,2018-02-27,3,10013531,1.00,"17,800.00"
4,Y2NgaWlpYA==,2018-02-27,4,10015613,1.00,"5,600.00"
...,...,...,...,...,...,...
993507,amxpamJk,2018-05-20,993507,10225636,0.18,"51,500.00"
993508,amxpbWNk,2018-04-11,993508,10016399,2.00,"57,700.00"
993509,amxpbWNk,2018-04-11,993509,10016834,1.00,"97,000.00"
993510,amxpbWNk,2018-04-11,993510,10018739,1.00,"82,500.00"


In [6]:
df['value'] = df.salesquantity * df.price

In [7]:
df = df.groupby(['date', 'csn']).agg(article= ('article', 'count'),
                                salesquantity = ('salesquantity', 'sum'),
                                value = ('value', 'sum')).reset_index()
df = df.sort_values(['csn', 'date'])
df['prev_date'] = df.groupby('csn')['date'].shift()
df['days_btw_trans'] = (df['date'] - df['prev_date']).dt.days
df['days_left_in_month'] = df['date'].dt.daysinmonth - df['date'].dt.day
df['month'] = df.date.dt.month
df

,date,csn,article,salesquantity,value,prev_date,days_btw_trans,days_left_in_month,month
26063,2018-03-02,Y2NgaWJoYw==,1,2.00,"36,500.00",NaT,NaN,29,3
27787,2018-03-04,Y2NgaWJoYw==,2,5.00,"125,000.00",2018-03-02,2.00,27,3
23604,2018-02-27,Y2NgaWlpYA==,17,27.00,"1,423,700.00",NaT,NaN,1,2
38122,2018-03-16,Y2NgaWlpYA==,5,6.00,"668,000.00",2018-02-27,17.00,15,3
98161,2018-05-24,Y2NgaWlpYA==,4,9.00,"485,800.00",2018-03-16,69.00,7,5
...,...,...,...,...,...,...,...,...,...
46001,2018-03-24,amxpamJk,4,4.00,"135,000.00",2018-02-11,41.00,7,3
58005,2018-04-07,amxpamJk,2,4.00,"195,000.00",2018-03-24,14.00,23,4
69910,2018-04-21,amxpamJk,9,8.17,"291,183.00",2018-04-07,14.00,9,4
95809,2018-05-20,amxpamJk,12,14.34,"327,044.00",2018-04-21,29.00,11,5


In [8]:
def get_train_data(df, month):
    # get features based on whole 2 months of transaction
    data_2m = df.loc[df['month'].isin([month-2, month-1])]
    data_2m = data_2m.sort_values(['csn', 'month'])
    data_2m = data_2m.groupby(['csn']).agg({'article': ['sum', 'min', 'max'],
                                      'salesquantity': ['sum', 'min', 'max'],
                                      'value': ['sum', 'min', 'max'],
                                      'days_btw_trans': ['mean', 'min', 'max'],
                                      'days_left_in_month': ['last']})
    data_2m.columns = ['_2m_'.join(gp) for gp in data_2m.columns.values]
    data_2m = data_2m.reset_index()
    # get features based on last month transaction
    data_1m = df.loc[df['month'].isin([month-1])]
    data_1m = data_1m.sort_values(['csn', 'month'])
    data_1m = data_1m.groupby(['csn']).agg({'article': ['sum', 'min', 'max'],
                                      'salesquantity': ['sum', 'min', 'max'],
                                      'value': ['sum', 'min', 'max'],
                                      'days_btw_trans': ['mean', 'min', 'max']})
    data_1m.columns = ['_1m_'.join(gp) for gp in data_1m.columns.values]
    data_1m = data_1m.reset_index()
    #
    if month == 7:
        data = data_2m.merge(data_1m, how='left', on='csn')
        data = data.fillna(0)
#         test = test.append(data, ignore_index=True)
        return data
    else:
        data_label = df.loc[df['month'].isin([month])]
        data_label = data_label.groupby("csn")["article"].sum().reset_index()
        #
        data = data_2m.merge(data_1m, how='left', on='csn')
        data = data.merge(data_label, how='left', on='csn')
        data['label'] = np.where(data['article'] > 0, 'Y', 'N')
        data = data.drop(columns=['article'])
        data = data.fillna(0)
#         train = train.append(data, ignore_index=True)
        print(data[data.label == 'Y'].shape[0], data[data.label == 'N'].shape[0])
        return data

In [9]:
# get train, test data. hardcode by month
train = pd.DataFrame()
test = pd.DataFrame()
for i in range(4, 8):
    if i == 7:
        test = test.append(get_train_data(df, i), ignore_index=True)
    else:
        train = train.append(get_train_data(df, i), ignore_index=True)
train.to_csv('train.csv', index=False)
test.to_csv('test.csv', index=False)
train

6578 6706
6416 6105
6396 6317


,csn,article_2m_sum,article_2m_min,article_2m_max,salesquantity_2m_sum,salesquantity_2m_min,salesquantity_2m_max,value_2m_sum,value_2m_min,value_2m_max,days_btw_trans_2m_mean,days_btw_trans_2m_min,days_btw_trans_2m_max,days_left_in_month_2m_last,article_1m_sum,article_1m_min,article_1m_max,salesquantity_1m_sum,salesquantity_1m_min,salesquantity_1m_max,value_1m_sum,value_1m_min,value_1m_max,days_btw_trans_1m_mean,days_btw_trans_1m_min,days_btw_trans_1m_max,label
0,Y2NgaWJoYw==,3,1,2,7.00,2.00,5.00,"161,500.00","36,500.00","125,000.00",2.00,2.00,2.00,27,3.00,1.00,2.00,7.00,2.00,5.00,"161,500.00","36,500.00","125,000.00",2.00,2.00,2.00,N
1,Y2NgaWlpYA==,22,5,17,33.00,6.00,27.00,"2,091,700.00","668,000.00","1,423,700.00",17.00,17.00,17.00,15,5.00,5.00,5.00,6.00,6.00,6.00,"668,000.00","668,000.00","668,000.00",17.00,17.00,17.00,N
2,Y2NgamRpZA==,39,1,16,47.86,2.03,20.14,"1,476,144.00","80,917.00","387,318.00",9.80,3.00,18.00,9,8.00,1.00,7.00,13.29,2.03,11.26,"319,797.01","80,917.00","238,880.01",12.00,10.00,14.00,N
3,Y2NgamVj,3,3,3,4.00,4.00,4.00,"33,400.00","33,400.00","33,400.00",0.00,0.00,0.00,6,3.00,3.00,3.00,4.00,4.00,4.00,"33,400.00","33,400.00","33,400.00",0.00,0.00,0.00,N
4,Y2Ngb2FpYg==,1,1,1,0.23,0.23,0.23,"132,217.00","132,217.00","132,217.00",0.00,0.00,0.00,6,1.00,1.00,1.00,0.23,0.23,0.23,"132,217.00","132,217.00","132,217.00",0.00,0.00,0.00,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38513,amxocWRl,12,12,12,17.84,17.84,17.84,"282,340.00","282,340.00","282,340.00",11.00,11.00,11.00,19,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,N
38514,amxpa2Rr,61,9,17,99.57,12.03,32.00,"1,270,664.00","214,336.00","357,265.00",14.00,7.00,21.00,5,27.00,10.00,17.00,41.27,17.00,24.27,"592,265.00","235,000.00","357,265.00",17.50,14.00,21.00,Y
38515,amxpaWZo,10,10,10,12.93,12.93,12.93,"477,605.00","477,605.00","477,605.00",0.00,0.00,0.00,8,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,N
38516,amxpamJk,23,2,12,26.51,4.00,14.34,"813,227.00","195,000.00","327,044.00",19.00,14.00,29.00,11,12.00,12.00,12.00,14.34,14.34,14.34,"327,044.00","327,044.00","327,044.00",29.00,29.00,29.00,N
